# 코로나 전,후 영화 산업 동향 분석을 통한 영화 산업 추이 예측

## 데이터 수집 및 가공 그리고 시각화를 통해 4년간의 영화 산업  동향분석
## 코로나 이전과 코로나 시기 그리고 현재의 영화 데이터 분석을 통해 엔데믹 이후 영화 산업 예측


### 분석할 데이터

- 박스오피스 (월별) 
- 좌석점유율 (기간별 - 월별)
- 장르분포 & 점유율 통계 (월별)
- 네이버 키워드에서 영화 순위(영화에 대한 관심도)
- 코로나 거리두기 시행이 영화 산업에 미치는 영향 (영화관 시간제한 / 좌석수) 
- 월별 코로나 확진자
- OTT 이용자 

**코로나 기준 : 정부 기관에서 발표한 시기 \
코로나 전 : 18년 4월 ~ 20년 3월 (2년)\
코로나 중 : 20년 4월 ~ 22년 3월 (2년)\
코로나 후 : 22년 4월 ~ 22년 6월 (3달)\


# 데이터 크롤링
- seleninum 사용

In [ ]:
# 셀레니움 import
from selenium import webdriver # 셀레니움을 통해 웹 브라우저를 컨트롤
from selenium.webdriver.common.by import By # 셀레니움 안에서 Html 속성 확인
from selenium.webdriver.common.keys import Keys # 키보드, 마우스 입력키
from selenium.webdriver.chrome.service import Service # 자동설정
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from webdriver_manager.chrome import ChromeDriverManager # 자동설정
import time
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json
import urllib
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
driver = webdriver.Chrome('./chromedriver')
# 설정한 url로 데이터 get함수로 요청
url = 'https://www.kobis.or.kr/kobis/business/stat/boxs/findPeriodSeatTicketList.do'
driver.get(url)

In [ ]:
# 지역 선택
driver.find_element(By.XPATH, '//*[@id="wideareaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="wideareaCd"]/option[3]').click()

In [ ]:
# 날짜 선택 후 조회 엑셀 저장
driver.find_element(By.XPATH, '//*[@id="endDate"]').clear()
driver.find_element(By.XPATH, '//*[@id="endDate"]').send_keys('2018-05-31')
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="startDate"]').clear()
driver.find_element(By.XPATH, '//*[@id="startDate"]').send_keys('2018-05-01')
driver.find_element(By.XPATH, '//*[@id="wideareaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/div[5]/button').click()
time.sleep(3)
driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/div/a').click()
time.sleep(3)
# wait 클래스 불러들이기 및 정의
from selenium.webdriver.support.ui import WebDriverWait
wait = WebDriverWait(driver, timeout=10, poll_frequency=1)

# alert 상태를 빼주는 함수들이 모두 클래스화 되어서 전달이 되어야 하네요
from selenium.webdriver.support import expected_conditions

alert = wait.until(expected_conditions.alert_is_present())
alert.accept()

In [ ]:
import os

path = './miniproject_movie/'
file_list = [file_nm for file_nm in os.listdir(path) if file_nm.endswith('xls')]

for index, file_nm in enumerate(file_list):
    print(index, file_nm)
    


# 데이터 가공

## 1. 지역별 월별로 따로 떨어져 있는 엑셀 데이터 합치기
- 좌석 점유율 (기간별 - 월별) \
- 지역: 전라북도 ~ 세종시

In [ ]:
import pandas as pd
data= pd.read_excel(f'data/KOBIS_기간별_좌석점유율_2022-07-09.xlsx', engine='openpyxl',skiprows=1,skipfooter = 1)

data['date']='2022-06'    # 날짜 열 추가
data['place'] = '전라북도'  # 지역 열 추가
data.head()

,순위,영화명,개봉일,좌석판매율,좌석점유율,좌석수,매출액,누적매출액,관객수,누적관객수,date,place
0,1.0,범죄도시 2,2022-05-18,0.186,0.009,730846.0,1.369618e+09,1.262375e+11,136151.0,12216638.0,2022-06,전라북도
1,2.0,마녀(魔女) Part2. The Other One,2022-06-15,0.171,0.006,482410.0,8.177053e+08,2.495603e+10,82740.0,2425343.0,2022-06,전라북도
2,2.0,쥬라기 월드: 도미니언,2022-06-01,0.145,0.006,468689.0,6.757527e+08,2.910183e+10,68153.0,2823283.0,2022-06,전라북도
3,2.0,탑건: 매버릭,2022-06-22,0.126,0.006,458062.0,6.026590e+08,2.322440e+10,57843.0,2183800.0,2022-06,전라북도
4,5.0,브로커,2022-06-08,0.089,0.005,388825.0,3.343014e+08,1.236483e+10,34786.0,1236006.0,2022-06,전라북도


In [ ]:
# 함수만들어서 데이터 불러오고 취합하기
# 엑셀 파일 하나에 
def make_up(y,z,D_list,file):   #x:파일이름 y: 엑셀 파일 시작 숫자 z: data_list 위치
  place_list = ['전라북도', '전라남도','제주도','부산시', '대구시','대전시','울산시','인천시','광주시','세종시']  # 저장되있는 파일의 지역이 동일하게 연속되기 때문에 리스트로 만들어주었음
  start= pd.DataFrame()

  for i in range(0,10):
    a = pd.read_excel(f'{file}/KOBIS_기간별_좌석점유율_2022-07-10 ({i+y}).xlsx', engine='openpyxl',skiprows=4,skipfooter = 1) 
    a['date'] = D_list[z]     # 날짜 열 추가
    a['place'] = place_list[i]   # 지역 열 추가
    start = start.append(a)    # append로 취합
  start.to_csv(f'{D_list[z]}.csv', index='False')


# 21년 8월 ~ 22년 2월
date_list = ['2022-02','2022-01','2021-12','2021-11','2021-10','2021-09','2021-08']
for i in range(0,7):
  make_up((i+4)*10,i)   # y: 엑셀 파일 시작 숫자 z: data_list 위치

#옹일한 방식으로 월별 csv 파일 만들기

- 모든파일 합치기(df : total_data.csv)

In [ ]:
import os

path = './ddd/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝 # 파일명 끝이 csv 파일로 되어있는 파일만 리스트로 반환해라


## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i)
    df = pd.concat([df,data])
    
df = df.reset_index(drop = True)
df


,Unnamed: 0,순위,영화명,개봉일,좌석판매율,좌석점유율,좌석수,매출액,누적매출액,관객수,누적관객수,date,place
0,0,1.0,더 배트맨,2022-03-01,0.034,0.008,461552.0,155180460.0,9.133703e+09,15759.0,884123.0,2022-03,전라북도
1,1,2.0,이상한 나라의 수학자,2022-03-09,0.042,0.006,346791.0,132940220.0,4.628560e+09,14584.0,487694.0,2022-03,전라북도
2,2,3.0,뜨거운 피,2022-03-23,0.037,0.003,205838.0,73099820.0,2.643288e+09,7821.0,276791.0,2022-03,전라북도
3,3,3.0,문폴,2022-03-16,0.027,0.003,192799.0,50559340.0,1.913811e+09,5208.0,188268.0,2022-03,전라북도
4,4,5.0,극장판 주술회전 0,2022-02-17,0.048,0.002,117430.0,52533500.0,5.448349e+09,5694.0,551219.0,2022-03,전라북도
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49351,42,3.0,베이비티스,2020-10-22,0.011,0.000,724.0,74000.0,2.902357e+07,8.0,3498.0,2020-10,세종시
49352,43,3.0,포기와 베스,2020-10-06,0.019,0.000,408.0,233700.0,6.756900e+06,8.0,280.0,2020-10,세종시
49353,44,3.0,내 사랑 찾기,2020-10-22,0.007,0.000,516.0,27000.0,4.557300e+06,4.0,560.0,2020-10,세종시
49354,45,3.0,지옥의 오르페,2020-10-25,0.014,0.000,204.0,77000.0,2.218400e+06,3.0,90.0,2020-10,세종시


- df파일 가공


In [ ]:
data_df = df
data_df.head(3)

,Unnamed: 0,영화명,개봉일,좌석판매율,좌석점유율,좌석수,매출액,누적매출액,관객수,누적관객수,date,place
0,0,더 배트맨,2022-03-01,0.034,0.008,461552.0,155180460.0,9.133703e+09,15759.0,884123.0,2022-03,전라북도
1,1,이상한 나라의 수학자,2022-03-09,0.042,0.006,346791.0,132940220.0,4.628560e+09,14584.0,487694.0,2022-03,전라북도
2,2,뜨거운 피,2022-03-23,0.037,0.003,205838.0,73099820.0,2.643288e+09,7821.0,276791.0,2022-03,전라북도


In [ ]:
#필요없는 열 제거

data_df.drop(['Unnamed: 0','매출액','누적매출액','관객수','누적관객수'], axis = 1, inplace = True)
data_df.head(3)

,영화명,개봉일,좌석판매율,좌석점유율,좌석수,date,place
0,더 배트맨,2022-03-01,0.034,0.008,461552.0,2022-03,전라북도
1,이상한 나라의 수학자,2022-03-09,0.042,0.006,346791.0,2022-03,전라북도
2,뜨거운 피,2022-03-23,0.037,0.003,205838.0,2022-03,전라북도


In [ ]:
# 열 이름 재정의 (타 데이터와 통일 위해)
data_df.rename( columns = {'영화명': 'title', '개봉일':'open_date'},inplace=True)
data_df.rename( columns = {'좌석판매율': 'seat_rate', '좌석점유율':'seat_share','좌석수':'seat_qty','place':'region'},inplace=True)

In [ ]:
# 시간(년-월) 분리하기
data_df['year'] = data_df.date.str.split('-').str[0]
data_df['month'] = data_df.date.str.split('-').str[1]
data_df.head(3)

,title,open_date,seat_rate,seat_share,seat_qty,date,region,year,month
0,더 배트맨,2022-03-01,0.034,0.008,461552.0,2022-03,전라북도,2022,03
1,이상한 나라의 수학자,2022-03-09,0.042,0.006,346791.0,2022-03,전라북도,2022,03
2,뜨거운 피,2022-03-23,0.037,0.003,205838.0,2022-03,전라북도,2022,03


In [ ]:
# year, month 열 숫자형으로 변환 (문자 -> 숫자)
data_df.info()
data_df['year'] = pd.to_numeric(data_df['year'])
data_df['month'] = pd.to_numeric(data_df['month'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49356 entries, 0 to 49355
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       49356 non-null  object 
 1   open_date   44745 non-null  object 
 2   seat_rate   49356 non-null  float64
 3   seat_share  49356 non-null  float64
 4   seat_qty    49356 non-null  float64
 5   date        49356 non-null  object 
 6   region      49356 non-null  object 
 7   year        49356 non-null  object 
 8   month       49356 non-null  object 
dtypes: float64(3), object(6)
memory usage: 3.4+ MB


In [ ]:
# 열 순서 재배열
data_df = data_df[['year','month','region','title','open_date','seat_rate','seat_share','seat_qty']]
data_df.head(3)

,year,month,region,title,open_date,seat_rate,seat_share,seat_qty
0,2022,3,전라북도,더 배트맨,2022-03-01,0.034,0.008,461552.0
1,2022,3,전라북도,이상한 나라의 수학자,2022-03-09,0.042,0.006,346791.0
2,2022,3,전라북도,뜨거운 피,2022-03-23,0.037,0.003,205838.0


## 2. 영화 장르 데이터 
분포 & 점유율 통계 (월별) 데이터 가공

In [ ]:
path = './ddd/'
file_list = os.listdir(path)
f_l = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝 # 파일명 끝이 csv 파일로 되어있는 파일만 리스트로 반환해라

path = 'genre/genre/After/2018/'
df = pd.DataFrame()
for i in f_l:
    a =  pd.read_csv(path+i,encoding='cp949',skipfooter = 1)
    df  = pd.concat([df,a])

In [ ]:
#모든 파일 합치기 함수 만들기

def merge_ing(x):   
  path = f'genre/genre/ing/{x}/'
  file_list = os.listdir(path)
  file_list_py = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝

  file_list_py
  ## csv 파일들을 DataFrame으로 불러와서 concat

  y = pd.DataFrame()
  for i in file_list_py:
      data = pd.read_csv(path + i,encoding='cp949',skipfooter = 1)
      y = pd.concat([y,data])

      
  y = y.reset_index(drop = True)
  return y
    
df5 = merge_ing(2022)

In [ ]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      252 non-null    int64 
 1   월       252 non-null    int64 
 2   장르      252 non-null    object
 3   작품편수    252 non-null    object
 4   총 이용건수  252 non-null    object
 5   이용점유율   252 non-null    object
dtypes: int64(2), object(4)
memory usage: 11.9+ KB


In [ ]:
# 컬럼명 재정의
genre_total.rename({'년도':'year','월':'month','장르':'genre','작품편수':'number_movie','총 이용건수': 't_use','이용점유율':'share'
                    }, axis = 1, inplace = True)
genre_total.head(1)

,year,month,genre,number_movie,t_use,share
0,2018-01-01,9,액션,"2,106",1487260,24.8


In [ ]:
genre_total.to_csv('genre_total.csv')

## 좌석점유율 통합 데이터 (df3)

In [5]:
df3 = pd.read_csv('./seat_list.csv') # 윈도우의 경우 cp949

In [7]:
df3.head(3)

,year,month,region,title,open_date,seat_rate,seat_share,seat_qty
0,2018,3,서울시,지금 만나러 갑니다,2018-03-14,0.209,0.030,2671033.0
1,2018,3,서울시,리틀 포레스트,2018-02-28,0.177,0.026,2354947.0
2,2018,3,서울시,사라진 밤,2018-03-07,0.165,0.021,1861831.0
